In [1]:
from utils.bigquery_tools import buscar_coches_bq 

In [6]:
# --- Escenario 3: Datos de Prueba ---

filtros_scenario_3 = {
    # --- Provenientes de Preferencias / PostProc Perfil ---
    "transmision_preferida": "automático", # No aplicará filtro específico de transmisión
    
    # --- Inferidos por LLM Filtros (Ejemplos) ---
    "batalla_min": None,         # No es alto
    "indice_altura_interior_min": None,
    "tipo_mecanica": ["GASOLINA"], # No BEV, quizás prioriza potencia/par
    
    # --- Aplicados por PostProc Filtros ---
    "estetica_min": 1.0,         # Regla: valora_estetica='no'
    "premium_min": 5.0,          # Regla: apasionado_motor='sí'
    "singular_min": 5.0,         # Regla: apasionado_motor='sí'
    
    # --- Resultado de Cálculo Modo 1 (Contado) ---
    "modo_adquisicion_recomendado": "Contado", # 
    "precio_max_contado_recomendado": 30000.0, # 
    "cuota_max_calculada": None,
    
    # --- Resultado de RAG (Ejemplo para aventura extrema) ---
    "tipo_carroceria": ['TODOTERRENO', 'PICKUP','SUV', 'CROSSOVER'] 
}

# Pesos simulados: Alta importancia a aventura (traccion, reductoras, altura), premium, singular. Baja a estética.
# (Basado en Estetica=1, Premium=5, Singular=5; 
#  y Aventura Extrema NUEVO: altura=8, traccion=10, reductoras=8. Suma Raw = 37)
pesos_scenario_3 = { 
    "estetica": 1.0 / 37.0,  # ~0.027
    "premium":  5.0 / 37.0,  # ~0.135
    "singular": 5.0 / 37.0,  # ~0.135
    "altura_libre_suelo": 8.0 / 37.0, # ~0.216 (Peso aventura 'extrema' NUEVO)
    "traccion":          10.0 / 37.0, # ~0.270 (Peso aventura 'extrema')
    "reductoras":         8.0 / 37.0, # ~0.216 (Peso aventura 'extrema' NUEVO)
}
# print(sum(pesos_scenario_3.values())) # Debería ser ~1.0

k_test_3 = 5
resultados_reales = None # Inicializar variable de resultados
resultados_reales = buscar_coches_bq(filtros=filtros_scenario_3, pesos=pesos_scenario_3, k=k_test_3)    
        

# --- PASO 4: Mostrar Resultados (si hubo y no hubo error previo) ---
if resultados_reales is not None: # Solo intentar mostrar si la búsqueda no falló y devolvió una lista
    if resultados_reales: # Si la lista no está vacía
        
        import pandas as pd
        df_resultados = pd.DataFrame(resultados_reales)
            # Limitar filas/columnas para mejor visualización inicial
        pd.set_option('display.max_rows', 10) 
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 1000)
            
            # Define las columnas que quieres ver y que esperas que existan
        columnas_a_mostrar = ['nombre', 'marca', 'tipo_mecanica', 'cambio_automatico' ,'traccion', 'reductoras', 'estetica'
                                  ,'premium', 'singular','tipo_carroceria', 'precio_compra_contado', 'score_total'] 
            # Comprobar qué columnas realmente existen en el DataFrame devuelto
        columnas_existentes = [col for col in columnas_a_mostrar if col in df_resultados.columns]
        # --- AÑADIR ESTE BLOQUE if/else CON EL PRINT ---
        if not columnas_existentes:
                 print("Resultados crudos (lista de dicts):", resultados_reales)
        else:
                 print(df_resultados[columnas_existentes].to_markdown(index=False)) 
            # --- FIN BLOQUE AÑADIDO ---

    else: # Si la lista de resultados está vacía
        print("No se encontraron resultados que coincidan con los filtros.")
else: # Si resultados_reales es None (porque hubo error en la búsqueda BQ)
     print("No se mostrarán resultados debido a un error previo en la búsqueda de BQ.")

# --- Fin del Código de Prueba ---

DEBUG:root:🔎 BigQuery SQL:

    SELECT
      -- Columnas que quieres devolver ...
      nombre, ID, marca, modelo, cambio_automatico, tipo_mecanica, 
      tipo_carroceria, indice_altura_interior, batalla, estetica, premium, 
      singular, altura_libre_suelo, traccion, reductoras, precio_compra_contado,
      
      -- --- INICIO CÁLCULO SCORE CON MIN-MAX SCALING ---
      (
        -- Estética (Rango Asumido: 1-10)
        COALESCE(SAFE_DIVIDE(COALESCE(estetica, 1.0) - 1.0, 10.0 - 1.0), 0) * @peso_estetica 
        
        -- Premium (Rango Asumido: 1-10)
        + COALESCE(SAFE_DIVIDE(COALESCE(premium, 1.0) - 1.0, 10.0 - 1.0), 0) * @peso_premium
        
        -- Singular (Rango Asumido: 1-10)
        + COALESCE(SAFE_DIVIDE(COALESCE(singular, 1.0) - 1.0, 10.0 - 1.0), 0) * @peso_singular
        
        -- Altura Libre Suelo (Ej: 67.0 - 438.0) <-- ¡USA TUS VALORES!
        + COALESCE(SAFE_DIVIDE(COALESCE(altura_libre_suelo, 67.0) - 67.0, 438.0 - 67.0), 0) * @peso_altura
        

DEBUG (BQ Search) ► Aplicando Filtro Económico Condicional...
DEBUG (BQ Search) ► Detectado Modo 1 Rec. Contado. Límite Precio: 30000.0
DEBUG (BQ Search) ► Añadiendo SQL: Precio Contado <= 30000.0
--- 🧠 SQL Query ---
 
    SELECT
      -- Columnas que quieres devolver ...
      nombre, ID, marca, modelo, cambio_automatico, tipo_mecanica, 
      tipo_carroceria, indice_altura_interior, batalla, estetica, premium, 
      singular, altura_libre_suelo, traccion, reductoras, precio_compra_contado,
      
      -- --- INICIO CÁLCULO SCORE CON MIN-MAX SCALING ---
      (
        -- Estética (Rango Asumido: 1-10)
        COALESCE(SAFE_DIVIDE(COALESCE(estetica, 1.0) - 1.0, 10.0 - 1.0), 0) * @peso_estetica 
        
        -- Premium (Rango Asumido: 1-10)
        + COALESCE(SAFE_DIVIDE(COALESCE(premium, 1.0) - 1.0, 10.0 - 1.0), 0) * @peso_premium
        
        -- Singular (Rango Asumido: 1-10)
        + COALESCE(SAFE_DIVIDE(COALESCE(singular, 1.0) - 1.0, 10.0 - 1.0), 0) * @peso_singular
    

DEBUG:urllib3.connectionpool:https://bigquery.googleapis.com:443 "POST /bigquery/v2/projects/thecarmentor-mvp2/jobs?prettyPrint=false HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/thecarmentor-mvp2/queries/fd0c364a-ea71-48c4-bbca-0a0d2d9e82c9?maxResults=0&location=EU&prettyPrint=false HTTP/1.1" 200 None
INFO:root:✅ BigQuery query ejecutada, 5 resultados obtenidos.


| nombre                                   | marca      | tipo_mecanica   | cambio_automatico   | traccion   | reductoras   |   estetica |   premium |   singular | tipo_carroceria   |   precio_compra_contado |   score_total |
|:-----------------------------------------|:-----------|:----------------|:--------------------|:-----------|:-------------|-----------:|----------:|-----------:|:------------------|------------------------:|--------------:|
| Land Rover Discovery Si6 HSE automatico  | Land Rover | GASOLINA        | True                | ALL        | True         |          7 |         8 |          5 | TODOTERRENO       |                   26750 |      0.795553 |
| Porsche Cayenne S Tiptronic              | Porsche    | GASOLINA        | True                | ALL        | False        |          8 |         9 |          6 | SUV               |                   26999 |      0.574488 |
| Porsche Cayenne Tiptronic S              | Porsche    | GASOLINA        | True                

In [4]:
# --- Escenario 4: Datos de Prueba ---

filtros_scenario_4 = {
    "transmision_preferida": "automático", 
    "batalla_min": None, # No era requisito explícito del perfil simulado
    "indice_altura_interior_min": None, # No era requisito explícito
    "tipo_mecanica": ["GASOLINA", "PHEVD", "PHEVG"], # Opciones familiares, no solo BEV
    "estetica_min": 1.0, # prioriza estética
    "premium_min": 1.0,  # No es apasionado
    "singular_min": 1.0, # No es apasionado
    "modo_adquisicion_recomendado": "Financiado", # Calculado en Modo 1
    "precio_max_contado_recomendado": None,
    "cuota_max_calculada": 450.0, # Calculado en Modo 1 (Ej: 54k ingresos, 8 años)
    "tipo_carroceria": ['SUV', 'MONOVOLUMEN', 'AUTOCARAVANA'] # RAG para familia+aventura ocasional
}

# Pesos simulados: Prioridad a factores de aventura ocasional (altura, tracción), 
# bajo peso a estética/premium/singular/reductoras. 
# Asumiendo usuario no alto (pesos bajos para batalla/índice).
# Raw: estetica=1, premium=1, singular=1. Aventura Ocasional: altura=6, traccion=4, reductoras=1. Batalla=0.5, Indice=0.5. Total=15.
pesos_scenario_4 = { 
    "estetica": 1.0 / 15.0,  # ~0.067
    "premium":  1.0 / 15.0,  # ~0.067
    "singular": 1.0 / 15.0,  # ~0.067
    "altura_libre_suelo": 6.0 / 15.0, # 0.4 (Peso aventura 'ocasional')
    "traccion":           4.0 / 15.0, # ~0.267 (Peso aventura 'ocasional')
    "reductoras":         1.0 / 15.0, # ~0.067 (Peso aventura 'ocasional')
    "batalla":            0.5 / 15.0, # ~0.033 (Peso bajo si no es alto)
    "indice_altura_interior": 0.5 / 15.0, # ~0.033 (Peso bajo si no es alto)
}
# print(sum(pesos_scenario_4.values())) # Debe ser ~1.0

k_test_4 = 5 # Pedir algunos resultados

# --- Llamada a la función (en tu notebook) ---
# resultados_4 = buscar_coches_bq(filtros=filtros_scenario_4, pesos=pesos_scenario_4, k=k_test_4)

resultados_reales = None # Inicializar variable de resultados
resultados_reales = buscar_coches_bq(filtros=filtros_scenario_4, pesos=pesos_scenario_4, k=10)    
        
if resultados_reales is not None: # Solo intentar mostrar si la búsqueda no falló y devolvió una lista
    if resultados_reales: # Si la lista no está vacía
        
        import pandas as pd
        df_resultados = pd.DataFrame(resultados_reales)
            # Limitar filas/columnas para mejor visualización inicial
        pd.set_option('display.max_rows', 10) 
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 1000)
            
            # Define las columnas que quieres ver y que esperas que existan
        columnas_a_mostrar = ['nombre', 'marca', 'tipo_mecanica', 'cambio_automatico' ,'traccion', 'reductoras', 'estetica'
                                  ,'premium', 'singular','tipo_carroceria', 'precio_compra_contado', 'score_total'] 
            # Comprobar qué columnas realmente existen en el DataFrame devuelto
        columnas_existentes = [col for col in columnas_a_mostrar if col in df_resultados.columns]
        # --- AÑADIR ESTE BLOQUE if/else CON EL PRINT ---
        if not columnas_existentes:
                 print("Resultados crudos (lista de dicts):", resultados_reales)
        else:
                 print(df_resultados[columnas_existentes].to_markdown(index=False)) 
            # --- FIN BLOQUE AÑADIDO ---

    else: # Si la lista de resultados está vacía
        print("No se encontraron resultados que coincidan con los filtros.")
else: # Si resultados_reales es None (porque hubo error en la búsqueda BQ)
     print("No se mostrarán resultados debido a un error previo en la búsqueda de BQ.")



DEBUG:root:🔎 BigQuery SQL:

    SELECT
      -- Columnas que quieres devolver ...
      nombre, ID, marca, modelo, cambio_automatico, tipo_mecanica, 
      tipo_carroceria, indice_altura_interior, batalla, estetica, premium, 
      singular, altura_libre_suelo, traccion, reductoras, precio_compra_contado,
      
      -- --- INICIO CÁLCULO SCORE CON MIN-MAX SCALING ---
      (
        -- Estética (Rango Asumido: 1-10)
        COALESCE(SAFE_DIVIDE(COALESCE(estetica, 1.0) - 1.0, 10.0 - 1.0), 0) * @peso_estetica 
        
        -- Premium (Rango Asumido: 1-10)
        + COALESCE(SAFE_DIVIDE(COALESCE(premium, 1.0) - 1.0, 10.0 - 1.0), 0) * @peso_premium
        
        -- Singular (Rango Asumido: 1-10)
        + COALESCE(SAFE_DIVIDE(COALESCE(singular, 1.0) - 1.0, 10.0 - 1.0), 0) * @peso_singular
        
        -- Altura Libre Suelo (Ej: 67.0 - 438.0) <-- ¡USA TUS VALORES!
        + COALESCE(SAFE_DIVIDE(COALESCE(altura_libre_suelo, 67.0) - 67.0, 438.0 - 67.0), 0) * @peso_altura
        

DEBUG (BQ Search) ► Aplicando Filtro Económico Condicional...
DEBUG (BQ Search) ► Detectado Modo 1 Rec. Financiado. Límite Cuota: 450.0
DEBUG (BQ Search) ► Añadiendo SQL: Cuota Estimada <= 450.0
--- 🧠 SQL Query ---
 
    SELECT
      -- Columnas que quieres devolver ...
      nombre, ID, marca, modelo, cambio_automatico, tipo_mecanica, 
      tipo_carroceria, indice_altura_interior, batalla, estetica, premium, 
      singular, altura_libre_suelo, traccion, reductoras, precio_compra_contado,
      
      -- --- INICIO CÁLCULO SCORE CON MIN-MAX SCALING ---
      (
        -- Estética (Rango Asumido: 1-10)
        COALESCE(SAFE_DIVIDE(COALESCE(estetica, 1.0) - 1.0, 10.0 - 1.0), 0) * @peso_estetica 
        
        -- Premium (Rango Asumido: 1-10)
        + COALESCE(SAFE_DIVIDE(COALESCE(premium, 1.0) - 1.0, 10.0 - 1.0), 0) * @peso_premium
        
        -- Singular (Rango Asumido: 1-10)
        + COALESCE(SAFE_DIVIDE(COALESCE(singular, 1.0) - 1.0, 10.0 - 1.0), 0) * @peso_singular
      

DEBUG:urllib3.connectionpool:https://bigquery.googleapis.com:443 "POST /bigquery/v2/projects/thecarmentor-mvp2/jobs?prettyPrint=false HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/thecarmentor-mvp2/queries/9f247a87-fbd7-412e-b2e7-488d26d05a22?maxResults=0&location=EU&prettyPrint=false HTTP/1.1" 200 None
INFO:root:✅ BigQuery query ejecutada, 10 resultados obtenidos.


| nombre                                                        | marca      | tipo_mecanica   | cambio_automatico   | traccion   | reductoras   |   estetica |   premium |   singular | tipo_carroceria   |   precio_compra_contado |   score_total |
|:--------------------------------------------------------------|:-----------|:----------------|:--------------------|:-----------|:-------------|-----------:|----------:|-----------:|:------------------|------------------------:|--------------:|
| Porsche Cayenne S Tiptronic                                   | Porsche    | GASOLINA        | True                | ALL        | False        |          8 |         9 |          6 | SUV               |                   26999 |      0.60492  |
| Porsche Cayenne Tiptronic S                                   | Porsche    | GASOLINA        | True                | ALL        | False        |          8 |         9 |          6 | SUV               |                   24999 |      0.602402 |
| Porsche Ca

In [3]:
# --- Escenario 5: Datos de Prueba ---

filtros_scenario_5 = {
    "transmision_preferida": "automático", 
    "batalla_min": None,
    "indice_altura_interior_min": None,
    "tipo_mecanica": ["BEV", "PHEVD", "PHEVG", "GASOLINA"], # Opciones eficientes/urbanas
    "estetica_min": 5.0, # Regla: valora_estetica='sí'
    "premium_min": 1.0,  # Regla: apasionado_motor='no'
    "singular_min": 1.0, # Regla: apasionado_motor='no'
    # --- Filtro económico directo del usuario (Modo 2) ---
    # OJO: La función buscar_coches_bq actualmente solo mira las claves 
    # 'precio_max_contado_recomendado' y 'cuota_max_calculada'.
    # Para que este filtro funcione, necesitarías:
    # OPCIÓN 1 (Preferida): Modificar buscar_coches_bq para que también revise 
    #                     'cuota_max' si 'modo_adquisicion_recomendado' no está.
    # OPCIÓN 2 (Workaround para esta prueba): "Engañar" a la función poniendo el valor 
    #                                       en la clave que sí lee. ¡NO ideal a largo plazo!
    "modo_adquisicion_recomendado": "Financiado", # <-- Simular esto para activar el filtro de cuota
    "cuota_max_calculada": 300.0, # <-- Poner aquí la cuota MÁXIMA del usuario
    "precio_max_contado_recomendado": None, 
    # "cuota_max": 300.0 # <-- El valor real estaría aquí en el estado 'economia'
    # "entrada": 2000.0 # <-- El valor real estaría aquí en el estado 'economia'
    # ------------------------------------------------------
    "tipo_carroceria": ['2VOL', 'SUV', 'COUPE'] # RAG para estética/urbano
}

# Pesos simulados: Prioridad MUY ALTA a estética. Baja a todo lo demás (aventura nula, no apasionado).
# Raw: estetica=5, premium=1, singular=1. Aventura Ninguna: altura=0, traccion=0, reductoras=0. No alto: batalla=0.5, indice=0.5. Total=8.
pesos_scenario_5 = { 
    "estetica": 5.0 / 8.0,  # 0.625
    "premium":  1.0 / 8.0,  # 0.125
    "singular": 1.0 / 8.0,  # 0.125
    "altura_libre_suelo": 0.0 / 8.0, # 0.0 (Aventura 'ninguna')
    "traccion":           0.0 / 8.0, # 0.0 (Aventura 'ninguna')
    "reductoras":         0.0 / 8.0, # 0.0 (Aventura 'ninguna')
    "batalla":            0.5 / 8.0, # 0.0625 (Peso bajo si no es alto)
    "indice_altura_interior": 0.5 / 8.0, # 0.0625 (Peso bajo si no es alto)
}
# print(sum(pesos_scenario_5.values())) # Debe ser 1.0

resultados_reales = None # Inicializar variable de resultados
resultados_reales = buscar_coches_bq(filtros=filtros_scenario_5, pesos=pesos_scenario_5, k=5
                                     )    
        
if resultados_reales is not None: # Solo intentar mostrar si la búsqueda no falló y devolvió una lista
    if resultados_reales: # Si la lista no está vacía
        
        import pandas as pd
        df_resultados = pd.DataFrame(resultados_reales)
            # Limitar filas/columnas para mejor visualización inicial
        pd.set_option('display.max_rows', 10) 
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 1000)
            
            # Define las columnas que quieres ver y que esperas que existan
        columnas_a_mostrar = ['nombre', 'marca', 'tipo_mecanica', 'cambio_automatico' ,'traccion', 'reductoras', 'estetica'
                                  ,'premium', 'singular','tipo_carroceria', 'precio_compra_contado', 'score_total'] 
            # Comprobar qué columnas realmente existen en el DataFrame devuelto
        columnas_existentes = [col for col in columnas_a_mostrar if col in df_resultados.columns]
        # --- AÑADIR ESTE BLOQUE if/else CON EL PRINT ---
        if not columnas_existentes:
                 print("Resultados crudos (lista de dicts):", resultados_reales)
        else:
                 print(df_resultados[columnas_existentes].to_markdown(index=False)) 
            # --- FIN BLOQUE AÑADIDO ---

    else: # Si la lista de resultados está vacía
        print("No se encontraron resultados que coincidan con los filtros.")
else: # Si resultados_reales es None (porque hubo error en la búsqueda BQ)
     print("No se mostrarán resultados debido a un error previo en la búsqueda de BQ.")



DEBUG:google.auth._default:Checking /Users/andresrsalamanca/.config/gcloud/application_default_credentials.json for explicit credentials as part of auth process...
DEBUG:google.auth._default:Explicit credentials path /Users/andresrsalamanca/.config/gcloud/application_default_credentials.json is the same as Cloud SDK credentials path, fall back to Cloud SDK credentials flow...
DEBUG:google.auth._default:Checking Cloud SDK credentials as part of auth process...
DEBUG:google.auth._default:Checking /Users/andresrsalamanca/.config/gcloud/application_default_credentials.json for explicit credentials as part of auth process...
DEBUG:google.auth._default:Explicit credentials path /Users/andresrsalamanca/.config/gcloud/application_default_credentials.json is the same as Cloud SDK credentials path, fall back to Cloud SDK credentials flow...
DEBUG:google.auth._default:Checking Cloud SDK credentials as part of auth process...
DEBUG:root:🔎 BigQuery SQL:

    SELECT
      -- Columnas que quieres dev

DEBUG (BQ Search) ► Aplicando Filtro Económico Condicional...
DEBUG (BQ Search) ► Detectado Modo 1 Rec. Financiado. Límite Cuota: 300.0
DEBUG (BQ Search) ► Añadiendo SQL: Cuota Estimada <= 300.0
--- 🧠 SQL Query ---
 
    SELECT
      -- Columnas que quieres devolver ...
      nombre, ID, marca, modelo, cambio_automatico, tipo_mecanica, 
      tipo_carroceria, indice_altura_interior, batalla, estetica, premium, 
      singular, altura_libre_suelo, traccion, reductoras, precio_compra_contado,
      
      -- --- INICIO CÁLCULO SCORE CON MIN-MAX SCALING ---
      (
        -- Estética (Rango Asumido: 1-10)
        COALESCE(SAFE_DIVIDE(COALESCE(estetica, 1.0) - 1.0, 10.0 - 1.0), 0) * @peso_estetica 
        
        -- Premium (Rango Asumido: 1-10)
        + COALESCE(SAFE_DIVIDE(COALESCE(premium, 1.0) - 1.0, 10.0 - 1.0), 0) * @peso_premium
        
        -- Singular (Rango Asumido: 1-10)
        + COALESCE(SAFE_DIVIDE(COALESCE(singular, 1.0) - 1.0, 10.0 - 1.0), 0) * @peso_singular
      

DEBUG:urllib3.connectionpool:https://bigquery.googleapis.com:443 "POST /bigquery/v2/projects/thecarmentor-mvp2/jobs?prettyPrint=false HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/thecarmentor-mvp2/queries/2d4c1661-1685-4eca-998e-cbc0a60826fe?maxResults=0&location=EU&prettyPrint=false HTTP/1.1" 200 None
INFO:root:✅ BigQuery query ejecutada, 5 resultados obtenidos.


| nombre                                            | marca          | tipo_mecanica   | cambio_automatico   | traccion   | reductoras   |   estetica |   premium |   singular | tipo_carroceria   |   precio_compra_contado |   score_total |
|:--------------------------------------------------|:---------------|:----------------|:--------------------|:-----------|:-------------|-----------:|----------:|-----------:|:------------------|------------------------:|--------------:|
| Jaguar E-Pace P200 AWD automatico                 | Jaguar         | GASOLINA        | True                | ALL        | False        |          8 |         8 |          5 | SUV               |                   19000 |      0.684352 |
| Jaguar XK Coupé 4.2 V8 automatico                 | Jaguar         | GASOLINA        | True                | RWD        | False        |          8 |         8 |          5 | COUPE             |                   19999 |      0.677742 |
| Alfa Romeo GT 2.0 16V JTS 16V Sportiva Sel